# Predicting EPL Result Using Poisson Distribution and Classification
Poisson Distribution Wiki: [Link](https://en.wikipedia.org/wiki/Poisson_distribution)

## Notebook Contents:

[Objective](#Objective)  
[Data Prep/Summary](#DPS)  
[Exploratory Data Analysis](#EDA)  
[Poisson Distribution](#PD)

<a id='Objective'></a>
### Objective:

Predict the probability of each matches results of English Premier League using past season data using Poisson distribution.<br>
We are not doing any statistical modeling here. We simply predict the probability of how many goals home/away team would score in every single game in season.<br>

<a id='DPS'></a>
### Data Prep/Summary:
- English Premier League Stats (2012 - Present)
- English Premier League Fixtures (2012 - Present)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For Poisson distribution
from scipy.stats import poisson

In [14]:
# Read past seasons data which have been cleaned beforehands
# We want teamnames, total number of goals each team scored and allowed ('Team', 'HGF', 'HGA')
# We calculate each teams offensive and defensive rating ('H_Att', 'A_Att', 'H_Def', 'A_Def')
# Number of yellow cards, red cards, and Discipline points and the number of clean sheets each team for the last season

# IMPORTANT
# table_13 has the final result of 13/14 season, with 12/13 seasons summary
# We will be able to predict the result using features 

table_13 = pd.read_csv('./Data/epl_table_1314.csv')[['Team', 'HGF', 'HGA', 'H_Att', 'A_Att', 'H_Def', 'A_Def', 'YC', 'RC', 'DIS', 'CS']]
table_14 = pd.read_csv('./Data/epl_table_1415.csv')[['Team', 'HGF', 'HGA', 'H_Att', 'A_Att', 'H_Def', 'A_Def', 'YC', 'RC', 'DIS', 'CS']]
table_15 = pd.read_csv('./Data/epl_table_1516.csv')[['Team', 'HGF', 'HGA', 'H_Att', 'A_Att', 'H_Def', 'A_Def', 'YC', 'RC', 'DIS', 'CS']]
table_16 = pd.read_csv('./Data/epl_table_1617.csv')[['Team', 'HGF', 'HGA', 'H_Att', 'A_Att', 'H_Def', 'A_Def', 'YC', 'RC', 'DIS', 'CS']]
table_17 = pd.read_csv('./Data/epl_table_1718.csv')[['Team', 'HGF', 'HGA', 'H_Att', 'A_Att', 'H_Def', 'A_Def', 'YC', 'RC', 'DIS', 'CS']]

table = [table_13, table_14, table_15, table_16, table_17]

In [16]:
# Read all season fixtures from 13/14 to 18/19 (present)
# We need home/away team and the final result to compare with our prediction
# We want how many goals they score to make useful EDAs

epl_detail_1314 = pd.read_csv('./Data/epl14.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]
epl_detail_1415 = pd.read_csv('./Data/epl15.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]
epl_detail_1516 = pd.read_csv('./Data/epl16.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]
epl_detail_1617 = pd.read_csv('./Data/epl17.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]
epl_detail_1718 = pd.read_csv('./Data/epl18.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]
epl_detail_1819 = pd.read_csv('./Data/epl19.csv')[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]

fixture = [epl_detail_1415, epl_detail_1516, epl_detail_1617, epl_detail_1718, epl_detail_1819]

<a id='PD'></a>
### Poisson Distribution
Poisson process, which is essentially a way to model random events at a known rate, we generate two Poisson distributions around those scores. Those give us the likelihood that each team will score no goals, one goal, two goals, etc. (Source:[How Our 2018 World Cup Predictions Work (FiveThirtyEight)](https://fivethirtyeight.com/features/how-our-2018-world-cup-predictions-work/))

In [7]:
def result_percentage(dataframe, hometeam, awayteam):
    '''
    returns array of size 2.
    list of percentage of hometeam win/away win/draw
    dataframe of percentage of how many goals each team scores in 90 minutes (min: 0, max: 5)
    '''
    home_avg = dataframe['HGF'].sum()/380
    away_avg = dataframe['HGA'].sum()/380
    
    home_score = float(dataframe[dataframe['Team'] == hometeam]['H_Att']) * float(dataframe[dataframe['Team'] == awayteam]['A_Def']) * home_avg
    away_score = float(dataframe[dataframe['Team'] == awayteam]['A_Att']) * float(dataframe[dataframe['Team'] == hometeam]['H_Def']) * away_avg

    score = []
    # maximum score for a team is 5
    for goals in range(0, 6):
        scores = {}
        scores['Home'] = (poisson.pmf(goals, home_score)) # Hometeam score
        scores['Away'] = (poisson.pmf(goals, away_score)) # away score

        if len(scores) == 2:
                    score.append(scores)

    score = pd.DataFrame(score, columns=(['Home', 'Away']))

    # % of home team winning
    # home score > away score
    # home[1] * away[0]
    # home[2] * away[0] + home[2] * away[1]
    # home[3] * away[0] + home[3] * away[1] + home[3] * away[2]
    # home[4] * away[0] + home[4] * away[1] + home[4] * away[2] + home[4] * away[3]
    # home[5] * away[0] + home[5] * away[1] + home[5] * away[2] + home[5] * away[3]+  home[5] * away[4]
    home_w = 0
    away_w = 0
    draw = 0
    result = []
    for home in range(1, len(score)):
        for away in range(0, home):
            home_w += (score['Home'][home] * score['Away'][away])
    result.append(home_w)

    for away in range(1, len(score)):
        for home in range(0, away):
            away_w += (score['Home'][home] * score['Away'][away])
    result.append(away_w)

    for home in range(0, len(score)):
        away = home
        draw += (score['Home'][home] * score['Away'][away])
    result.append(draw)

    return result, score

In [15]:
# Percentage of the number goals in 17/18 Arsenal(Home) vs Chelsea(Away)
mat = result_percentage(table_17, 'Arsenal', 'Chelsea')[1]

In [16]:
# 6 x 6 Matrix form
result = []
for i in mat['Home']:
    for j in mat['Away']:
        result.append(round(i * j, 4))
result = np.reshape(result, (6,6))
df_ = pd.DataFrame(result, index=range(0,6), columns=('0', '1', '2', '3', '4', '5'))
df_.index.names = ['Home']
df_.columns.names = ['Away']

In [17]:
# Total Probability of W/D/L
temp = pd.DataFrame(result_percentage(table_17, 'Arsenal', 'Chelsea')[0], index=('Home Win', 'Away Win', 'Draw')).T

In [18]:
mat

,Home,Away
0,0.116639,0.213280
1,0.250619,0.329550
2,0.269249,0.254601
3,0.192842,0.131132
4,0.103589,0.050655
5,0.044516,0.015654


In [19]:
df_

Away,0,1,2,3,4,5
Home,,,,,,
0,0.0249,0.0384,0.0297,0.0153,0.0059,0.0018
1,0.0535,0.0826,0.0638,0.0329,0.0127,0.0039
2,0.0574,0.0887,0.0686,0.0353,0.0136,0.0042
3,0.0411,0.0636,0.0491,0.0253,0.0098,0.0030
4,0.0221,0.0341,0.0264,0.0136,0.0052,0.0016
5,0.0095,0.0147,0.0113,0.0058,0.0023,0.0007


In [20]:
temp.rename(index={0: "Probability"})

,Home Win,Away Win,Draw
Probability,0.493166,0.272024,0.207251


In [21]:
# Getting the final result of a game
def decision(result_p):
    if result_p[2] > result_p[1] and result_p[2] > result_p[0]:
        return 'D'
    elif result_p[0] > result_p[1] and result_p[0] > result_p[2]:
        return 'H'
    elif result_p[1] > result_p[0] and result_p[1] > result_p[2]:
        return 'A'

In [22]:
# Predict each game
def predict(fixture, rating):
    for i in range(len(fixture)):
        if fixture['HomeTeam'][i] in list(rating['Team']) and fixture['AwayTeam'][i] in list(rating['Team']):
            result = result_percentage(rating, fixture['HomeTeam'][i], fixture['AwayTeam'][i])[0]
            fixture.loc[i, 'Predict'] = decision(result)
            fixture.loc[i, 'W'] = result[0]
            fixture.loc[i, 'D'] = result[2]
            fixture.loc[i, 'L'] = result[1]

    fixture.loc[:, 'Result'] = (fixture.loc[:, 'FTR'] == fixture.loc[:, 'Predict']) * 1

In [23]:
for i in range(len(fixture)):
    predict(fixture[i], table[i])

In [35]:
print("EPL 14/15 Prediction Score:", epl_detail_1415.dropna()['Result'].mean())
print("EPL 15/16 Prediction Score:", epl_detail_1516.dropna()['Result'].mean())
print("EPL 16/17 Prediction Score:", epl_detail_1617.dropna()['Result'].mean())
print("EPL 17/18 Prediction Score:", epl_detail_1718.dropna()['Result'].mean())
print("EPL 18/19 Prediction Score:", epl_detail_1819.dropna()['Result'].mean())

print("Average Score with Poisson Model:", (epl_detail_1819.dropna()['Result'].mean() + epl_detail_1415.dropna()['Result'].mean() + 
 epl_detail_1516.dropna()['Result'].mean() + epl_detail_1617.dropna()['Result'].mean() + (epl_detail_1718.dropna()['Result'].mean())) / 5)


EPL 14/15 Prediction Score: 0.5
EPL 15/16 Prediction Score: 0.4375
EPL 16/17 Prediction Score: 0.5257352941176471
EPL 17/18 Prediction Score: 0.5110294117647058
EPL 18/19 Prediction Score: 0.6046511627906976
Average Score with Poisson Model: 0.51578317373461


In [41]:
epl_detail_1819.dropna().reset_index()[['HomeTeam', 'AwayTeam', 'W', 'D', 'L', 'FTR', 'Predict', 'Result']]

,HomeTeam,AwayTeam,W,D,L,FTR,Predict,Result
0,Manchester Utd,Leicester City,0.738472,0.138564,0.072993,H,H,1
1,Huddersfield,Chelsea,0.113782,0.204240,0.667711,A,A,1
2,Newcastle Utd,Tottenham,0.203776,0.275398,0.517535,A,A,1
3,Arsenal,Manchester City,0.215050,0.203304,0.556066,A,A,1
4,Liverpool,West Ham Utd,0.758934,0.077210,0.032043,H,H,1
5,Southampton,Burnley,0.235455,0.292911,0.469577,D,A,0
6,Chelsea,Arsenal,0.583343,0.239465,0.169422,H,H,1
7,Everton,Southampton,0.512122,0.259121,0.223675,H,H,1
8,West Ham Utd,Bournemouth,0.400191,0.266698,0.329019,A,H,0
9,Burnley,Watford,0.400948,0.342053,0.256443,A,H,0


In [28]:
test = pd.concat([epl_detail_1415.dropna(), epl_detail_1516.dropna(), epl_detail_1617.dropna(), epl_detail_1718.dropna()], ignore_index=True)
test

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,Predict,W,D,L,Result
0,Arsenal,Crystal Palace,H,2,1,H,0.670188,0.238443,0.085600,1
1,Manchester Utd,Swansea City,A,1,2,H,0.460424,0.266210,0.269164,0
2,Stoke City,Aston Villa,A,0,1,H,0.573491,0.258294,0.163386,0
3,West Bromwich,Sunderland,D,2,2,H,0.399652,0.260440,0.334923,0
4,West Ham Utd,Tottenham,A,0,1,A,0.308918,0.253250,0.431572,1
5,Liverpool,Southampton,H,2,1,H,0.632460,0.195279,0.149472,1
6,Newcastle Utd,Manchester City,A,0,2,A,0.123377,0.164193,0.667755,1
7,Aston Villa,Newcastle Utd,D,0,0,A,0.331225,0.266375,0.398268,0
8,Crystal Palace,West Ham Utd,A,1,3,D,0.320397,0.354387,0.324857,0
9,Everton,Arsenal,D,2,2,H,0.497412,0.216091,0.266437,0


In [120]:
epl_detail_1819.dropna().head()

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,Predict,W,D,L,Result
0,Manchester Utd,Leicester City,H,2,1,H,0.738472,0.138564,0.072993,1
3,Huddersfield,Chelsea,A,0,3,A,0.113782,0.204240,0.667711,1
4,Newcastle Utd,Tottenham,A,1,2,A,0.203776,0.275398,0.517535,1
7,Arsenal,Manchester City,A,0,2,A,0.215050,0.203304,0.556066,1
8,Liverpool,West Ham Utd,H,4,0,H,0.758934,0.077210,0.032043,1


In [158]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

knn = KNeighborsClassifier(n_neighbors=5)
rf = RandomForestClassifier()
lr = LogisticRegression()
X = test.drop(columns=(['HomeTeam', 'AwayTeam', 'FTR','Result','Predict', 'FTHG', 'FTAG']))
y = test['FTR']
knn.fit(X, y)
rf.fit(X, y)
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [159]:
X.columns

Index(['W', 'D', 'L'], dtype='object')

In [164]:
predict_X = epl_detail_1819.drop(columns=(['HomeTeam', 'AwayTeam', 'FTR','Result','Predict','FTHG','FTAG'])).dropna()
compare = epl_detail_1819.dropna()[['HomeTeam', 'AwayTeam']]
compare = compare.reindex(columns=compare.columns.tolist() + ['Actual', 'Predict'])
compare['Actual'] = epl_detail_1819['FTR']
compare['Predict'] = rf.predict(predict_X)

In [165]:
compare.loc[:, 'Result'] = (compare.loc[:, 'Actual'] == compare.loc[:, 'Predict']) * 1

In [166]:
compare

,HomeTeam,AwayTeam,Actual,Predict,Result
0,Manchester Utd,Leicester City,H,H,1
3,Huddersfield,Chelsea,A,A,1
4,Newcastle Utd,Tottenham,A,A,1
7,Arsenal,Manchester City,A,D,0
8,Liverpool,West Ham Utd,H,H,1
9,Southampton,Burnley,D,D,1
11,Chelsea,Arsenal,H,H,1
12,Everton,Southampton,H,H,1
15,West Ham Utd,Bournemouth,A,H,0
17,Burnley,Watford,A,H,0


In [167]:
compare['Result'].mean()

0.5116279069767442